In [1]:
import numpy as np
import sys

INPUT_NEURON = 4
HID_NEURON = 3
OUTPUT_NEURON = 4

class Neurocontroller:
    def __init__(self, learning_rate=0.2):
        self.weights_0_1 = np.random.uniform(-0.5, 0.5, (5, 3)) #равномерное распределение
        self.weights_1_2 = np.random.uniform(-0.5, 0.5, (4, 4)) #равномерное распределение
        self.sigmoid_mapper = np.vectorize(self.sigmoid) #Позволяет пробежаться по вектору, и к каждому элементу применить сигмоидную функцию
        self.learning_rate = np.array([learning_rate])
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_derivative(self, val):
        return ( val * (1 - val) )

    #Прямой проход
    def predict(self, inputs):
        #Вычисляем вход в скрытый слой и выход из него
        inputs = np.append(inputs, 1)#Дополнительный вход для смещения
        inputs_1 = np.dot(inputs,self.weights_0_1) #Умножаем веса на входные значения
        outputs_1 = self.sigmoid_mapper(inputs_1)
        #Вычисляем вход в выходной слой и выход из него
        outputs_1 = np.append(outputs_1, 1)#Дополнительный вход для смещения
        inputs_2 = np.dot(outputs_1,self.weights_1_2)
        outputs_2 = self.sigmoid_mapper(inputs_2)
        return outputs_2

    def fit(self, inputs, target_predict):
        
        #Вычисляем вход в скрытый слой и выход из него
        inputs = np.append(inputs, 1)#Дополнительный вход для смещения
        inputs_1 = np.dot(inputs,self.weights_0_1) #Умножаем веса на входные значения
        outputs_1 = self.sigmoid_mapper(inputs_1)
        
        #Вычисляем вход в выходной слой и выход из него
        outputs_1  = np.append(outputs_1, 1)#Дополнительный вход для смещения
        inputs_2 = np.dot(outputs_1,self.weights_1_2)
        outputs_2 = self.sigmoid_mapper(inputs_2)
        fact_predict_out = outputs_2

        error_out = np.array([])
        #Вычислить ошибку для выходного слоя
        for o in range(OUTPUT_NEURON):
            error_out = np.append(error_out,(target_predict[o] - fact_predict_out[o]) * self.sigmoid_derivative(fact_predict_out[o]))
        
        sum = 0
        error_hid = np.array([])
        #Вычислить ошибку для срытого слоя
        for i in range(HID_NEURON):
            temp_sum = error_out * self.weights_1_2[i,:] # Пройтись по строкам
            sum = np.sum(temp_sum)
            error_hid = np.append(error_hid, self.sigmoid_derivative(outputs_1[i]) * sum);
        
        
        
        #Обновить веса для соединений скрытый слой - выходной слой
        #обновить смещения для нейронов выходного слоя
        #выходы из скрытого : outputs_1
        #oшибка выходного слоя : error_out
        for i in range(OUTPUT_NEURON):
            self.weights_1_2[:,i]+=self.learning_rate*error_out[i]*outputs_1
        
        #Обновить веса для входной слой - скрытого слоя
        #Выходы из входного слоя inputs
        #Обновить смещения для нейронов скрытого слоя
        for i in range(HID_NEURON):
            self.weights_0_1[:,i]+=self.learning_rate*error_hid[i]*inputs

In [2]:
def MSE(d, y):
    return 0.5 * np.sum((d-y)**2)
train_data = [
    ([2, 0, 0, 0], [0,0,1,0]),
    ([2,0,0,1], [0,0,1,0]),
    ([2,0,1,1], [1,0,0,0]),
    ([2,0,1,2], [1,0,0,0]),
    ([2,1,0,2], [0,0,0,1]),
    ([2,1,0,1], [1,0,0,0]),
    ([1,0,0,0], [0,0,1,0]),
    ([1,0,0,1], [0,0,0,1]),
    ([1,0,1,1], [1,0,0,0]),
    ([1,0,1,2], [0,0,0,1]),
    ([1,1,0,2], [0,0,0,1]),
    ([1,1,0,1], [0,0,0,1]),
    
    ([0,0,0,0], [0,0,1,0]),
    ([0,0,0,1], [0,0,0,1]),
    ([0,0,1,1], [0,0,0,1]),
    ([0,0,1,2], [0,1,0,0]),
    ([0,1,0,2], [0,1,0,0]),
    ([0,1,0,1], [0,0,0,1]),
]

#атаковать бежать бродить/уворачиваться прятаться
action = ["Attack", "Run", "Wander", "Hide"]


In [9]:
#Обучение и тестирование нейроконтроллера
epochs = 500
learning_rate = 0.2
network = Neurocontroller(learning_rate=learning_rate)

for ep in range(epochs):
    inputs_ = []
    correct_predictions = []
    for input_stat, correct_predict in train_data:
        
        #Ошибка на входном векторе
        i_loss = MSE(network.predict(np.array(input_stat)), np.array(correct_predict))
        #sys.stdout.write("\r {}, i_loss: {}".format(str(100 * ep/float(epochs))[:4], str(i_loss)[:5]))
        
        #Train
        network.fit(np.array(input_stat).T, np.array(correct_predict))
        
        inputs_.append(np.array(input_stat))
        correct_predictions.append(np.array(correct_predict))
      
    #print(correct_predictions)
    #Считаем ошибку после эпохи
    train_loss = 0
    for m in range(len(inputs_)):
        train_loss += MSE(network.predict(np.array(inputs_[m])), np.array(correct_predictions[m]))
    sys.stdout.write("\rProgress: {}, Training loss: {}".format(str(100 * ep/float(epochs))[:4], str(train_loss)[:5]))
        


Progress: 99.8, Training loss: 0.504

In [20]:
#Проверяем сеть
for input_stat, correct_predict in train_data:
    print("For input: {} the prediction is: {}, expected: {}".format(
        str(input_stat),
        str(action[network.predict(np.array(input_stat)).argmax()]),
        str(action[np.array(correct_predict).argmax()])))

For input: [2, 0, 0, 0] the prediction is: Wander, expected: Wander
For input: [2, 0, 0, 1] the prediction is: Wander, expected: Wander
For input: [2, 0, 1, 1] the prediction is: Attack, expected: Attack
For input: [2, 0, 1, 2] the prediction is: Attack, expected: Attack
For input: [2, 1, 0, 2] the prediction is: Hide, expected: Hide
For input: [2, 1, 0, 1] the prediction is: Attack, expected: Attack
For input: [1, 0, 0, 0] the prediction is: Wander, expected: Wander
For input: [1, 0, 0, 1] the prediction is: Hide, expected: Hide
For input: [1, 0, 1, 1] the prediction is: Attack, expected: Attack
For input: [1, 0, 1, 2] the prediction is: Hide, expected: Hide
For input: [1, 1, 0, 2] the prediction is: Hide, expected: Hide
For input: [1, 1, 0, 1] the prediction is: Hide, expected: Hide
For input: [0, 0, 0, 0] the prediction is: Wander, expected: Wander
For input: [0, 0, 0, 1] the prediction is: Hide, expected: Hide
For input: [0, 0, 1, 1] the prediction is: Hide, expected: Hide
For inpu

In [21]:
#Прогон на тестовых данных
#атаковать бежать бродить прятаться
action = ["Attack", "Run", "Wander", "Hide"]
test_data = [
    ([2, 1, 1, 1], [1,0,0,0]),
    ([1,1,1,2], [0,0,0,1]),
    ([0,0,0,0], [0,0,1,0]),
    ([0,1,1,1], [0,0,0,1]),
    ([2,0,1,3], [0,0,0,1]),
    ([2,1,0,3], [0,0,0,1]),
    ([0,1,0,3], [0,1,0,0]),
    
]
for input_stat, correct_predict in test_data:
    print("For input: {} the prediction is: {}, expected: {}".format(
        str(input_stat), 
        str(action[network.predict(np.array(input_stat)).argmax()]),
        str(action[np.array(correct_predict).argmax()])))
#         str(network.direct_distribution(np.array(input_stat))),
#         str(np.array(correct_predict))))

For input: [2, 1, 1, 1] the prediction is: Attack, expected: Attack
For input: [1, 1, 1, 2] the prediction is: Hide, expected: Hide
For input: [0, 0, 0, 0] the prediction is: Wander, expected: Wander
For input: [0, 1, 1, 1] the prediction is: Hide, expected: Hide
For input: [2, 0, 1, 3] the prediction is: Hide, expected: Hide
For input: [2, 1, 0, 3] the prediction is: Hide, expected: Hide
For input: [0, 1, 0, 3] the prediction is: Run, expected: Run


In [13]:
len(train_data)

18

In [2]:
import numpy as np
import sys
weights_0_1 =  np.random.uniform(-0.5, 0.5, (5,3))
print(weights_0_1)
inputs = np.array([1, 2, 3,4,5])
inputs_1 = np.dot(inputs,weights_0_1)
print(inputs_1)

[[ 0.10761236 -0.40999681 -0.08251098]
 [ 0.13300704  0.49957307  0.25933433]
 [ 0.08303307  0.36907027 -0.1481706 ]
 [ 0.01834851 -0.31871495 -0.47966446]
 [ 0.26472631  0.32282889  0.10889235]]
[ 2.01975127  2.0356448  -1.38255021]


In [4]:
l = [ 0.10761236, 0.13300704,0.08303307,0.01834851,0.26472631]
inputs = np.array([1, 2, 3,4,5])
l2 = l*inputs
l2.sum()

2.0197512399999997

In [102]:

np.append(inputs, 11)


array([ 1,  2,  3,  4,  5, 11])

In [109]:
def sigmoid_derivative(val):
    return ( val * (1 - val) )

target_predict = np.array([1,1,1,1])
fact_predict_out = np.array([2,3,4,5])
(target_predict - fact_predict_out) * sigmoid_derivative(fact_predict_out)

array([ 2, 12, 36, 80])

In [15]:
n = np.array([2,0,1,1])
n = np.append(n, 1) 
print(n)
weights_0_1 = np.random.uniform(-0.5, 0.5, (5, 3))
inputs_1 = np.dot(n,weights_0_1)
inputs_1


[2 0 1 1 1]


array([ 0.70986221,  0.57366845, -1.55359659])

In [19]:
def MSE(d, y):
    return 0.5 * np.sum((d-y)**2)
d = np.array([0, 0, 1, 0])
y = np.array([2, 3, 4, 5])
#-2 -3 -3 -5
# 4  9  9  25
#
MSE(d,y)

23.5

In [4]:
import numpy as np
weights_0_1 = np.random.uniform(-0.5, 0.5, (5, 3))
print(weights_0_1)
weights_0_1[:,0]+=5*5*np.array([1,1,1,1,1])
print(weights_0_1)

[[ 0.24109519 -0.28518315 -0.30845519]
 [-0.25665082 -0.04776185 -0.20033632]
 [-0.20289359 -0.21702956 -0.30258498]
 [ 0.16585842  0.13127477  0.33070176]
 [-0.13932579 -0.41562836 -0.24051136]]
[[25.24109519 -0.28518315 -0.30845519]
 [24.74334918 -0.04776185 -0.20033632]
 [24.79710641 -0.21702956 -0.30258498]
 [25.16585842  0.13127477  0.33070176]
 [24.86067421 -0.41562836 -0.24051136]]
